In [28]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
%matplotlib inline

In [29]:
# DATA INGESTION

# Fetch the data
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [30]:
# Check the DataFrame
train_df.tail(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
881,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,NaN,S
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q


In [31]:
# PREPROCESSING

# Create a fake Survived column for test data
test_df.loc[:,"Survived"] = -1

In [32]:
# Concatenate both training and test data
data = pd.concat([train_df,test_df]).reset_index(drop=True)

In [33]:
# FEATURE ENGINEERING

# Drop non interested columns
data.drop(['PassengerId','Name','Cabin','Ticket'], axis=1, inplace=True)
data.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,NaN,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [34]:
# Feature engineering of Age 

# Impute missing values in the entire 'Age' & 'Fare' columns using median
data['Age']=data['Age'].fillna(data['Age'].median(skipna=True)) #skipna=True to exclude NaN values when computing the result
#data['Minor']=np.where(data['Age']<=16, 1, 0)

In [35]:
data['Fare']=data['Fare'].fillna(data['Fare'].mean(skipna=True))
# Cap the Fare at a reasonable threshold, e.g., 300
data['Fare'] = data['Fare'].clip(upper=300)

In [36]:
# Fill in missing values with the most common embarkation value - 'S' * Parch
data["Embarked"].fillna(data['Embarked'].value_counts().idxmax(), inplace=True)
# Embarking from the most common location or not
data['Embarked_S'] = np.where(data['Embarked'] == 'S', 1, 0)
# Fill in missing values with the most common parent-child number
data["Parch"].fillna({'Parch': data['Parch'].value_counts().idxmax()}, inplace=True)

/var/folders/jv/67_frsw54hd0ldzsgf5xrjvw0000gn/T/ipykernel_1873/283054969.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Embarked"].fillna(data['Embarked'].value_counts().idxmax(), inplace=True)
/var/folders/jv/67_frsw54hd0ldzsgf5xrjvw0000gn/T/ipykernel_1873/283054969.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which

In [37]:
data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked      0
Embarked_S    0
dtype: int64

In [38]:
data.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Embarked_S
0,0,3,male,22.0,1,0,7.2500,S,1
1,1,1,female,38.0,1,0,71.2833,C,0
2,1,3,female,26.0,0,0,7.9250,S,1
3,1,1,female,35.0,1,0,53.1000,S,1
4,0,3,male,35.0,0,0,8.0500,S,1
5,0,3,male,28.0,0,0,8.4583,Q,0
6,0,1,male,54.0,0,0,51.8625,S,1
7,0,3,male,2.0,3,1,21.0750,S,1
8,1,3,female,27.0,0,2,11.1333,S,1
9,1,2,female,14.0,1,0,30.0708,C,0


In [39]:
# Join two features to simplify modelling
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1  # +1 to account for the individual

In [40]:

# Drop the columns
data.drop(['Embarked', 'SibSp', 'Parch'], axis=1, inplace=True) #,'SibSp','Parch'

In [41]:

# Convert categorical variables or the ones that represent such ('Pclass') to dummy/numerical variables (one-hot encoding)
data = pd.get_dummies(data, columns=['Sex','Pclass'], dtype=int, drop_first=True) # drop_first to avoid collinearity #'Embarked', 'Sex', 'AgeGroup','Pclass'

In [42]:
data.head(10)

,Survived,Age,Fare,Embarked_S,FamilySize,Sex_male,Pclass_2,Pclass_3
0,0,22.0,7.2500,1,2,1,0,1
1,1,38.0,71.2833,0,2,0,0,0
2,1,26.0,7.9250,1,1,0,0,1
3,1,35.0,53.1000,1,2,0,0,0
4,0,35.0,8.0500,1,1,1,0,1
5,0,28.0,8.4583,0,1,1,0,1
6,0,54.0,51.8625,1,1,1,0,0
7,0,2.0,21.0750,1,5,1,0,1
8,1,27.0,11.1333,1,3,0,0,1
9,1,14.0,30.0708,0,2,0,1,0


In [43]:
# split the training and test data again
train_df = data[data.Survived != -1].reset_index(drop=True) 
test_df = data[data.Survived == -1].reset_index(drop=True)

In [44]:
test_df.head(5)

,Survived,Age,Fare,Embarked_S,FamilySize,Sex_male,Pclass_2,Pclass_3
0,-1,34.5,7.8292,0,1,1,0,1
1,-1,47.0,7.0000,1,2,0,0,1
2,-1,62.0,9.6875,0,1,1,1,0
3,-1,27.0,8.6625,1,1,1,0,1
4,-1,22.0,12.2875,1,3,0,0,1


In [45]:
# CROSS-VALIDATION
# We create a new column called kfold and fill it with -1
train_df['kfold'] = -1

# The next step is to randomize the rows of the data
train_df = train_df.sample(frac=1,random_state=32).reset_index(drop=True)

# Fetch the targets
y = train_df.Survived.values

# Inititate the kfold class
kf = model_selection.StratifiedKFold(n_splits=5)

# fill the new kfold column
for f, (t_, v_) in enumerate(kf.split(X=train_df,y=y)):
    train_df.loc[v_,'kfold'] = f

In [46]:
train_df.head()

,Survived,Age,Fare,Embarked_S,FamilySize,Sex_male,Pclass_2,Pclass_3,kfold
0,0,28.0,8.0500,1,1,1,0,1,0
1,1,28.0,133.6500,1,2,0,0,0,0
2,1,28.0,89.1042,0,2,0,0,0,0
3,0,28.0,7.7375,0,1,1,0,1,0
4,0,28.0,0.0000,1,1,1,1,0,0


In [ ]:
# Collect accuracies
lst = []

# Loop folds
for fold in range(0,5):
    # Training data is where kfold is not equal to provided fold
    df_train = train_df[train_df.kfold != fold].reset_index(drop=True)
    
    # Validation data is where kfold is equal to provided fold
    df_valid = train_df[train_df.kfold == fold].reset_index(drop=True)

    # Drop the Survived and kfold column from dataframe and convert it to a numpy array
    x_train = df_train.drop(['Survived','kfold'],axis=1).values
    y_train = df_train.Survived.values

    # Similarly, for validation
    x_valid = df_valid.drop(['Survived','kfold'],axis=1).values
    y_valid = df_valid.Survived.values

    scaler = StandardScaler() # Solved ConvergenceWarning
    x_train = scaler.fit_transform(x_train)
    x_valid = scaler.transform(x_valid)

    # INITIALIZE THE MODEL & FINE-TUNING
    model = LogisticRegression(max_iter=1000, random_state=32)
    # Fit the model on training data
    model.fit(x_train,y_train)

    # Create predictions for validations samples
    preds = model.predict(x_valid)

    # Calculate & print accuracy
    accuracy = metrics.accuracy_score(y_valid,preds)
    print(f"Fold = {fold}, Accuracy = {accuracy}")

    lst.append(accuracy)

Fold = 0, Accuracy = 0.776536312849162
Fold = 1, Accuracy = 0.7921348314606742
Fold = 2, Accuracy = 0.8033707865168539
Fold = 3, Accuracy = 0.8146067415730337
Fold = 4, Accuracy = 0.7865168539325843


In [48]:
Average = sum(lst) / len(lst) 
print(f"Average accuracy = {Average}")

Average accuracy = 0.7946331052664617


In [49]:
# Make predictions on the test data
#test_predictions = model.predict(test_df.drop('Survived',axis=1).values)

In [50]:
# Make predictions on the test data using predict_proba to get probabilities
test_probabilities = model.predict_proba(test_df.drop('Survived', axis=1).values)[:, 1]

# Set a custom threshold (e.g., 0.3) to predict class labels instead of default 0.5
custom_threshold = 0.3 # lower threshold - capturing more survivors; higher threshold - stricter about predicting survival.
test_predictions = (test_probabilities >= custom_threshold).astype(int)

In [51]:
# Prepare the submission file
submission = pd.read_csv('../data/submission.csv')
submission['Survived'] = test_predictions
submission.head(60)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,0
9,901,0


In [52]:
# Save to CSV
submission.to_csv('../data/submission.csv', index=False)
print("Submission file saved as 'submission.csv'.")

Submission file saved as 'submission.csv'.
